In [5]:
import tensorflow as tf
import os
      
IMAGE_WIDTH = 256
IMAGE_HEIGHT = 256
IMAGE_CHANNEL = 3

NUM_LABEL = 50

In [2]:
#Load Dataset
def _parse_image_function(example_proto):
    features = tf.io.parse_single_example(example_proto, image_feature_description)
    image = tf.io.decode_raw(features['image'], tf.uint8)
    image.set_shape([IMAGE_CHANNEL * IMAGE_WIDTH * IMAGE_HEIGHT])
    image = tf.reshape(image, [IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNEL])

    label = tf.cast(features['label'], tf.int32)
    label = tf.one_hot(label, NUM_LABEL)

    return image, label

def read_dataset(epochs, batch_size, channel, channel_name):

    filenames = [os.path.join(channel, channel_name + '.tfrecords')]
    dataset = tf.data.TFRecordDataset(filenames)

    image_feature_description = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'label': tf.io.FixedLenFeature([], tf.int64),
    }

    dataset = dataset.map(_parse_image_function, num_parallel_calls=10)
    dataset = dataset.prefetch(10)
    dataset = dataset.repeat(epochs)
    dataset = dataset.shuffle(buffer_size=10 * batch_size)
    dataset = dataset.batch(batch_size, drop_remainder=True)

    return dataset

In [ ]:
DATASET_PATH = '/raid/food_competition_KT_set1/kt_food.tfrecord'

dataset = read_dataset(10, 100, '/raid/food_competition_KT_set1', 'kt_food')
print(dataset)